### Basic Imports

In [69]:
import nbimporter
from typing import Dict, Tuple, List
import numpy as np
from tqdm import tqdm

### Env Vars

In [70]:
QUBITS_NUM = 4 
N = 16
K = 4
NUM_SHOTS = 1024
NUM_ITERATIONS = 10

w_vector = np.asarray([4,3,2,1])

### Simulator Backend

In [71]:
from qiskit import Aer
from qiskit.utils import QuantumInstance, algorithm_globals

seed = 50
algorithm_globals.random_seed = seed

simulator_backend = Aer.get_backend('qasm_simulator')

### BFGS Optimizer

In [72]:
# from qiskit.algorithms.optimizers import L_BFGS_B
from scipy.optimize import minimize

# bfgs_optimizer = L_BFGS_B(maxiter=NUM_ITERATIONS)

### K input states (computational basis)

In [73]:
from utiles import *

In [74]:
input_states = get_first_k_eigenvectors_from_n_computational_basis(K, N)

### Ansatz State

In [75]:
from ansatz_circuit_item2 import get_full_variational_quantum_circuit

In [76]:
init_circuit_params = {
    "thetas": np.random.uniform(low=0, high=2*np.pi, size=8),
    "phis": np.random.uniform(low=0, high=2*np.pi, size=4),
    "D1": 2,
    "D2": 8
}

In [77]:
def prepare_circuit_params(thetas) -> Dict:
     return {
    "thetas": thetas[4:],
    "phis": thetas[:4],
    "D1": 2,
    "D2": 8
     }

In [78]:
def get_ansatz_state(circuit_params, input_state):
    circuit_params_with_input_state = {**circuit_params, "input_state": input_state}
    return get_full_variational_quantum_circuit(**circuit_params_with_input_state)

## Expectation Value

### convert hamiltonian to pauli strings

In [79]:
#TODO: change the a_i and J_ij params into random values
from qiskit.opflow import X, Z, Y, I, H, S
from qiskit.opflow import ListOp

H_transverse_ising = 0.5*((I^I^I^X) + (I^I^X^I) + (I^X^I^I) + (X^I^I^I) + \
                          (Z^Z^I^I) + (Z^I^Z^I) + (Z^I^I^Z) + (I^Z^Z^I) + \
                          (I^Z^I^Z) + (I^I^Z^Z))

LiH_molecule_4_qubits = -7.49894690201071*(I^I^I^I) + \
-0.0029329964409502266*(X^X^Y^Y) + \
0.0029329964409502266*(X^Y^Y^X) + \
0.01291078027311749*(X^Z^X^I) + \
-0.0013743761078958677*(X^Z^X^Z) + \
0.011536413200774975*(X^I^X^I) + \
0.0029329964409502266*(Y^X^X^Y) + \
-0.0029329964409502266*(Y^Y^X^X) + \
0.01291078027311749*(Y^Z^Y^I) + \
-0.0013743761078958677*(Y^Z^Y^Z) + \
0.011536413200774975*(Y^I^Y^I) + \
0.16199475388004184*(Z^I^I^I) + \
0.011536413200774975*(Z^X^Z^X) + \
0.011536413200774975*(Z^Y^Z^Y) + \
0.12444770133137588*(Z^Z^I^I) + \
0.054130445793298836*(Z^I^Z^I) + \
0.05706344223424907*(Z^I^I^Z) + \
0.012910780273117487*(I^X^Z^X) + \
-0.0013743761078958677*(I^X^I^X) + \
0.012910780273117487*(I^Y^Z^Y) + \
-0.0013743761078958677*(I^Y^I^Y) + \
0.16199475388004186*(I^Z^I^I) + \
0.05706344223424907*(I^Z^Z^I) + \
0.054130445793298836*(I^Z^I^Z) + \
-0.013243698330265966*(I^I^Z^I) + \
0.08479609543670981*(I^I^Z^Z) + \
-0.013243698330265952*(I^I^I^Z) 

In [80]:
def transfrom_hamiltonian_into_pauli_string(hamiltonian) -> List:
    pauli_operators = hamiltonian.to_pauli_op().settings['oplist']
    pauli_coeffs = list(map(lambda pauli_operator: pauli_operator.coeff, pauli_operators))
    pauli_strings = list(map(lambda pauli_operator: pauli_operator.primitive, pauli_operators))
    return pauli_coeffs, pauli_strings

### pauli string reduction to sigma_z's

In [81]:
from qiskit.circuit.library.standard_gates import HGate, SGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

In [82]:
reducing_to_pauli_z_mapping = {
    'I': 'I',
    'Z': 'Z',
    'X': 'Z',
    'Y': 'Z'
} 

In [83]:
def reduce_pauli_matrixes_into_sigma_z(pauli_string) -> str:
    reduced_pauli_string = ""
    for matrix_index in range(QUBITS_NUM):
        pauli_matrix = str(pauli_string[matrix_index])
        reduced_pauli_matrix = reducing_to_pauli_z_mapping[pauli_matrix]
        reduced_pauli_string = reduced_pauli_matrix + reduced_pauli_string
    
    return reduced_pauli_string

In [84]:
def add_layer_of_gates_for_reducing_paulis_to_sigma_z(pauli_string, quantum_circuit):
    quantum_registers = QuantumRegister(QUBITS_NUM, name="qubit")
    additional_circuit_layer = QuantumCircuit(quantum_registers)
    
    for quantum_register_index, pauli_matrix in enumerate(pauli_string):
        if pauli_matrix == "X":
            additional_circuit_layer.append(HGate(), [quantum_registers[quantum_register_index]])
        if pauli_string == "Y":
            additional_circuit_layer.append(HGate(), [quantum_registers[quantum_register_index]])
            additional_circuit_layer.append(SGate(), [quantum_registers[quantum_register_index]])
                
    extended_quantum_circuit = quantum_circuit.compose(additional_circuit_layer)
    return extended_quantum_circuit

### probabilities distribution

In [85]:
def get_probability_distribution(counts: Dict) -> Dict:
    proba_distribution = {state: (count / NUM_SHOTS) for state, count in counts.items()}
    return proba_distribution

def calculate_probabilities_of_measurments_in_computational_basis(quantum_state_circuit) -> Dict:
    quantum_state_circuit.measure_all()
    
    transpiled_quantum_state_circuit = transpile(quantum_state_circuit, simulator_backend) 
    Qobj = assemble(transpiled_quantum_state_circuit)
    result = simulator_backend.run(Qobj).result()
    counts = result.get_counts(quantum_state_circuit)
    
    return get_probability_distribution(counts)

### Expectation value from probabilities

In [86]:
def sort_probas_dict_by_qubits_string_keys(proba_distribution: Dict) -> Dict:
    return dict(sorted(proba_distribution.items()))

def reset_power_of_minus_1(power_of_minus_1):
    power_of_minus_1 = 0
    return power_of_minus_1

def convert_pauli_string_into_str(pauli_string) -> str:
    return str(pauli_string)

def calculate_expectation_value_of_pauli_string_by_measurments_probas(pauli_string, ansatz_circuit):
    pauli_string_expectation_value = 0
    power_of_minus_1 = 0
    
    pauli_string_str = convert_pauli_string_into_str(pauli_string)
    extended_ansatz_circuit = add_layer_of_gates_for_reducing_paulis_to_sigma_z(pauli_string_str, ansatz_circuit)
    probas_distribution = calculate_probabilities_of_measurments_in_computational_basis(extended_ansatz_circuit)
    
    reduced_pauli_string = reduce_pauli_matrixes_into_sigma_z(pauli_string)
    sorted_probas_distribuition = sort_probas_dict_by_qubits_string_keys(probas_distribution)
    for qubits_string, proba in sorted_probas_distribuition.items():
        for string_index in range(QUBITS_NUM):
            if(str(qubits_string[string_index])=="1" and str(pauli_string[string_index])=="Z"):
                power_of_minus_1 += 1
            
        pauli_string_expectation_value += pow(-1, power_of_minus_1)*proba
        power_of_minus_1 = reset_power_of_minus_1(power_of_minus_1)
        
    return pauli_string_expectation_value

In [87]:
def get_expectation_value(ansatz_circuit, pauli_coeffs, pauli_strings):
    total_expection_value = 0
    
    for pauli_coeff, pauli_string in zip(pauli_coeffs, pauli_strings):
        total_expection_value += calculate_expectation_value_of_pauli_string_by_measurments_probas(
                                                                                    pauli_string, ansatz_circuit)
    
    return total_expection_value

## Objective Function

In [88]:
from qiskit import assemble, transpile

def cost_function(thetas, hamiltonian):
    L_w = 0
    circuit_params = prepare_circuit_params(thetas)
    computational_eigenvectors = get_first_k_eigenvectors_from_n_computational_basis(K, N)
    
    pauli_coeffs, pauli_strings = transfrom_hamiltonian_into_pauli_string(LiH_molecule_4_qubits)
    
    for j in tqdm(range(K)):
        ansatz_state = get_ansatz_state(circuit_params, computational_eigenvectors[j])
        L_w += w_vector[j]*get_expectation_value(ansatz_state, pauli_coeffs, pauli_strings)
        
    return L_w

### Optimization

In [89]:
initial_thetas = np.random.uniform(low=0, high=2*np.pi, size=12)
optimizer_result = minimize(cost_function,x0=initial_thetas,args=(LiH_molecule_4_qubits),method="BFGS",options={"maxiter":NUM_ITERATIONS})
print(optimizer_result)
optimal_thetas = prepare_circuit_params(optimizer_result.x)

computational_eigenvectors = get_first_k_eigenvectors_from_n_computational_basis(K, N)
optimal_ansatz_state = get_ansatz_state(optimal_thetas, computational_eigenvectors[K-1])
pauli_coeffs, pauli_strings = transfrom_hamiltonian_into_pauli_string(LiH_molecule_4_qubits)


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:12<00:00,  3.08s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:09<00:00,  2.44s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:12<00:00,  3.01s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.78s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:10<00:00,  2.73s/it]

      fun: 105.072265625
 hess_inv: array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])
      jac: array([ 8.99153920e+07,  3.44719360e+07,  4.15498240e+07, -8.78182400e+06,
        4.78412800e+07,  9.48961280e+07,  7.81189120e+07,  4.12876800e+07,
        1.02629376e+08,  1.30547712e+08,  8.80803840e+07,  5.70163200e+07])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 324
      nit: 0
     njev: 24
   status: 2
  success: False
        x: array([2.51140199, 5.4162110

In [90]:
print(get_expectation_value(optimal_ansatz_state, pauli_coeffs, pauli_strings))
print(optimal_thetas)

10.30859375
{'thetas': array([4.58438125, 4.81003888, 5.84610864, 3.63379255, 4.69853045,
       1.85215533, 5.85631752, 0.90149964]), 'phis': array([2.51140199, 5.41621105, 0.99989489, 5.4753456 ]), 'D1': 2, 'D2': 8}


In [91]:
pauli_coeffs, pauli_strings = transfrom_hamiltonian_into_pauli_string(LiH_molecule_4_qubits)
computational_eigenvectors = get_first_k_eigenvectors_from_n_computational_basis(K, N)
for eigenvalue_index, eigenvector in enumerate(computational_eigenvectors):
    print(eigenvector)
    optimal_ansatz_state = get_ansatz_state(optimal_thetas, eigenvector)
    print("eigenvalue for k =", eigenvalue_index)
    print(get_expectation_value(optimal_ansatz_state, pauli_coeffs, pauli_strings))

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
eigenvalue for k = 0
10.95703125
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
eigenvalue for k = 1
10.515625
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
eigenvalue for k = 2
10.083984375
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
eigenvalue for k = 3
10.4453125


In [92]:
from numpy import linalg as LA
print(LA.eig(LiH_molecule_4_qubits.to_matrix()))
eigen_values = LA.eig(LiH_molecule_4_qubits.to_matrix())[0]
print(sorted(eigen_values, reverse=True))

(array([-6.76981322+0.j, -7.71405669+0.j, -7.13040696+0.j, -7.56998474+0.j,
       -7.56998474+0.j, -7.13040696+0.j, -7.51199971+0.j, -7.78339621+0.j,
       -7.15152548+0.j, -7.70047584+0.j, -7.86277316+0.j, -7.51199971+0.j,
       -7.71405669+0.j, -7.78339621+0.j, -7.71405669+0.j, -7.36481744+0.j]), array([[ 1.00000000e+00+0.j,  1.83715697e-18+0.j, -0.00000000e+00+0.j,
        -0.00000000e+00+0.j, -0.00000000e+00+0.j, -0.00000000e+00+0.j,
        -0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
        -0.00000000e+00+0.j, -0.00000000e+00+0.j, -0.00000000e+00+0.j,
         0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j],
       [ 0.00000000e+00+0.j,  0.00000000e+00+0.j, -0.00000000e+00+0.j,
        -0.00000000e+00+0.j, -1.05567066e-01+0.j,  9.94412185e-01+0.j,
        -4.16108621e-18+0.j, -5.50591836e-20+0.j,  0.00000000e+00+0.j,
        -0.00000000e+00+0.j, -0.00000000e+00+0.j, -0.00000000e+00+0.j,
         0.00000000e+00+0.j,